In [ ]:
#Solução de equações lineares baseada no algoritmo quântico HHL
#Trabalho de Conclusão de Curso - Engenharia Elétrica
#Universidade de Caxias do Sul (UCS)
#Autor: Guilherme Adamatti Bridi

In [ ]:
from qiskit import *
from qiskit import Aer, execute, IBMQ
from qiskit import QuantumCircuit, QuantumRegister
from qiskit.tools.monitor import job_monitor
from qiskit.tools.visualization import plot_histogram
from qiskit.compiler import transpile, assemble
from qiskit.transpiler import PassManager
from qiskit.extensions import UnitaryGate
from qiskit.providers.aer import AerSimulator
from qiskit.aqua.utils.controlled_circuit import get_controlled_circuit
from qiskit.algorithms.linear_solvers.hhl import HHL
from qiskit.quantum_info import Statevector
from qiskit.algorithms.linear_solvers.numpy_linear_solver import NumPyLinearSolver
import numpy as np
from scipy import linalg

In [ ]:
#-----------------------Parâmetros de entrada-----------------------------

#Sistema 2x2
#B = np.array([[4, -2],
#              [-2, 4]])

#p = np.array([0.6, -0.8])

#Sistema 4x4
B =          np.array([[12, -2, 0, 0],
                       [-2, 12, -6, 0],
                       [0, -6, 12, -2],
                       [0, 0, -2, 12]])

p = np.array([0.7, -0.5, -0.5, -0.1])

DPtol = 10 #Desvio padrão admissível na representação binária, que pode introduzir erros teóricos na solução do sistema
NQmax = 6 #Número de q-bits aceitável no registrador alpha

#-----------------Preparação dos parâmetros do circuito-----------------

#Teste de validade das dimensões do sistema linear
if len(B) != len(p) or len(B) > 4:
    sys.exit("Dimensões inadequadas") 
    
eig = linalg.eig(B) #Autovalor da matriz B original
eig = np.abs(eig[0][0]) #Autovalor para complementar matrizes de ordem 3

#Adequação de sistemas de ordem 3
if np.ceil(np.log2(len(B))) > np.log2(len(B)):
    Baux = B
    paux = p
    B = eig * np.identity(pow(2, int(np.ceil(np.log2(len(B))))))
    p = np.zeros(pow(2, int(np.ceil(np.log2(len(B))))))
    B[0: len(Baux), 0: len(Baux)] = Baux
    p[0: len(Baux)] = paux    

#Normalização do sistema linear
B = B / linalg.norm(p)
p = p / linalg.norm(p) 

#Teste de hermiticidade
Bo = np.transpose(B)
for k in range(len(B)):
    for l in range(len(B)):
        if abs(B[k, l] - Bo[k, l]) > 0:
            sys.exit("Matriz não hermitiana")
    
nb = int(np.ceil(np.log2(len(p)))) #Número de q-bits do registrador beta

eigSave = linalg.eigh(B) 
eigVectors = eigSave[1] #Autovetores da matriz B
eigSave = np.sort(eigSave[0]) #Autovalores da matriz B

#Autovalores diagonalizados
eigMatrix = np.identity(len(eigSave))
for k in range(len(eigSave)):
    eigMatrix[k][k] = np.abs(eigSave[k])
    
#Verifica se todos os autovalores são positivos
for k in range(len(eigSave)):
    if eigSave[k] < 0:
        sys.exit("Autovalor negativo ou nulo")     

s = np.max(eigSave)/np.min(eigSave) #Número de condição
nk = int(np.floor((2**NQmax - 1) / s)) #Constante nk de controle que garante NQmax

#Rotina de acomodação dos autovalores
for k in range(nk):
    estadosSave = [0, 0, 0, 0]
    estadosUnique = [0, 0, 0, 0]
    eigUnique = [0, 0, 0, 0]
    estadosSave[0] = int(k + 1) #Posição do menor autovalor
    
    for l in range(2**nb - 1):
        estadosSave[l + 1] = int(np.around(estadosSave[0] * eigSave[l + 1] / eigSave[0])) #%Posição dos demais autovalores
    
    estadosUn = np.unique(estadosSave) #Elimina posições repetidas
    
    #Elimina posições 0
    g = 0
    for l in range(len(estadosUn)):
        if int(estadosUn[l]) != 0:
            estadosUnique[g] = int(estadosUn[l])
            g += 1        
    
    #Obtém os autovalores considerados únicos
    g = 0
    h = 1
    for l in range(2**nb):
        if estadosSave[l] == estadosUnique[g]:
            h = 1
            eigUnique[g] = eigSave[l]
            g += 1
        
        else:
            eigUnique[g - 1] = (h * eigUnique[g - 1] + eigSave[l]) / (h + 1)
            h += 1

    nrSave = g #Número de rotações controladas
    estadosUnique = estadosUnique[0 : nrSave] #Elimina posições 0
    eigUnique = eigUnique[0 : nrSave] #Elimina posições 0
    nqSave = int(np.floor(np.log2(estadosUnique[nrSave - 1] * 2))) #Número de q-bits no registrador alpha 
    
    #Cálculo da constante t para cada autovalor
    ti = np.zeros(2**nb)
    for l in range(2**nb):
        ti[l] = estadosSave[l] / (eigSave[l] * 2**nqSave) * 2 * np.pi        

    tSave = np.mean(ti) #t médio
    dpSave = 100 * np.std(ti) / tSave #Desvio padrão de t
    
    #Verifica se dp é menor igual ao aceitável
    if dpSave <= DPtol:
        #Armazena os parâmetros e quebra o laço
        dp = dpSave
        nr = nrSave
        nq = nqSave
        t = tSave
        estados = estadosUnique
        eig = eigUnique
        break
            
#Verifica se o erro está abaixo do tolerável
if dp > DPtol:
    sys.exit("Erro não tolerável")         

C = abs(np.amin(eigSave)) #C definido de forma maximizar a probabilidade de leitura 1 no q-bit ancilla

R = np.zeros(nr) 
for k in range(nr):
    estados[k] = bin(int(estados[k]))[2:].zfill(nq) #Estados em binário
    R[k] = 2 * np.arcsin(C / eig[k]) #Calcula ângulos de rotação original

#Matriz auxiliar de posições binárias
eigMat = np.zeros((nq, nr)) 
for k in range(nq):
    for l in range(nr):
        eigMat[k][l] = int(estados[l][k])

#Laço para obter R, P e controle
Po = np.zeros(nr)
controle = np.zeros(nr)
for k in range(nr):
        comp = eigMat[:, k]
        menor = 4

        for l in range(nq):
            cont = 0
            igual = np.zeros(nr)
            for g in range(nr):
                if eigMat[l][g] == comp[l] and g != k:
                    cont += 1
                    igual[g] -= R[k]

            if cont < menor: 
                menor = cont
                Po[k] = nq - l - 1
                Raux = igual
                controle[k] = eigMat[l][k]
                
        R += Raux
        

#Escrita dos pârametros        
print('nb =', nb)
print('nq =', nq)
print('nr =', nr)
print('t =', t)
print('Dp% =', dp, '%')
print('C =', C)
print('Autovalores =', abs(eigSave))
print('s =', s)
print('Autovalores unicos =', eig)
print('estados =', estados)
print('R =', R)
print('P =', Po)
print('controle =', controle)

In [ ]:
#-------------------Construção do circuito quântico-------------------------

#Registradores
alpha = QuantumRegister(nq, name = 'alpha')
beta = QuantumRegister(nb, name = 'beta')
ancilla = QuantumRegister(1, name = 'ancilla')
c = ClassicalRegister(1 + nb, name = 'c')

#Circuito
circuito = QuantumCircuit(beta, alpha, ancilla, c)

#Vetor p
circuito.initialize(p, beta)
#circuito.barrier()

#Hadamard
circuito.h(alpha)
#circuito.barrier()

#U-controlled
for k in range(nq):
    diag = np.identity(len(eigSave), dtype=complex)
    for l in range(len(eigSave)):
        diag[l][l] = np.exp(1j * eigMatrix[l][l] * t * 2**k)
        
    Bex = np.matmul(np.matmul(eigVectors, diag), np.transpose(eigVectors)) #Matriz exponenciada

    QR = QuantumRegister(nb)
    QC = QuantumCircuit(QR)
    QC.unitary(Bex, QR) #Unitary gate
    U = QC.to_gate().control(1) #Controlled gate
    
    if nb == 1: #Sistema 2x2
        circuito.append(U, [alpha[k], beta[0]]) #Append
        
    elif nb == 2: #Sistema 4x4
        circuito.append(U, [alpha[k], beta[0], beta[1]]) #Append
    
    #circuito.barrier()
    
#circuito.barrier()

#QTF'
for k in range(nq//2):
    circuito.swap(alpha[k], alpha[nq - k - 1])
    
for n in range(nq):
    for k in range(n, -1, -1):
        if k != n:
            circuito.cp(-np.pi/2**(n - k), alpha[k], alpha[n])          
    circuito.h(alpha[n])
    
#circuito.barrier()

#Rotação
for k in range(nr):
    if controle[k] == 0:
        circuito.x(alpha[int(Po[k])])

    circuito.cry(R[k], alpha[int(Po[k])], ancilla[0])

    if controle[k] == 0:
        circuito.x(alpha[int(Po[k])])     
    
    #circuito.barrier()
    
#circuito.barrier()
    
#QTF
for n in range(nq - 1, -1, -1):
    circuito.h(alpha[n])

    for k in range(n):
        circuito.cp(np.pi/2**(n - k), alpha[k], alpha[n])
        
for k in range(nq//2):
    circuito.swap(alpha[k], alpha[nq - k - 1])    

#circuito.barrier()
    
#U-controlled'
for k in range(nq):
    diag = np.identity(len(eigSave), dtype=complex)
    for l in range(len(eigSave)):
        diag[l][l] = np.exp(-1j * eigMatrix[l][l] * t * 2**k)
        
    Bex = np.matmul(np.matmul(eigVectors, diag), np.transpose(eigVectors)) #Matriz exponenciada
    
    QR = QuantumRegister(nb)
    QC = QuantumCircuit(QR)
    QC.unitary(Bex, QR) #Unitary gate
    U = QC.to_gate().control(1) #Controlled gate
    
    if nb == 1: #Sistema 2x2
        circuito.append(U, [alpha[k], beta[0]]) #Append
        
    elif nb == 2: #Sistema 4x4
        circuito.append(U, [alpha[k], beta[0], beta[1]]) #Append
        
    #circuito.barrier()
        
#circuito.barrier()

#Hadamard
circuito.h(alpha)
#circuito.barrier()

circuito.draw('mpl')

In [ ]:
#-------------------Solução pelo vetor de estado-------------------------

back_vector = Aer.get_backend('statevector_simulator') #statevector_simulator bakend
job_vector = execute(circuito, back_vector)
resultado = job_vector.result()
vector = resultado.get_statevector(circuito, decimals=10) #Vetor de estados

classica = np.array(linalg.solve(B, p)) #Solução clássica 
quantica = np.array(vector[2**(nq + nb): (2**(nq + nb) + 2**nb)]) / C #Solução quântica

erro = 100 * np.mean(np.abs((quantica - classica) / classica)) #Erro teórico
fidelidade = state_fidelity(quantica/ linalg.norm(quantica), classica / linalg.norm(classica)) #Fidelidade
probabilidade = 100 * np.abs(sum(quantica * quantica) * C**2) #Probabilidade de medida

#Escreve os resultados
print('Solução clássica:', classica)
print('Solução quântica:', quantica)
print('Erro teórico:', erro, '%')
print('Fidelidade:', fidelidade)
print('p(ancilla = 1):', probabilidade, '%')

In [ ]:
#-----------------------Medida-----------------------------

#Se o sistema for 2x2, se aplica o operador Hadamard no registrador alpha
if nb == 1:
    circuito.h(beta[0])

for k in range(nb): #Medida do registrador alpha
    circuito.measure(beta[k], c[k])

circuito.measure(ancilla, c[nb]) #Medida do registrador ancilla

circuito.draw('mpl')

In [ ]:
#-----------------------Transpilação do circuito-----------------------------

transp = transpile(circuito, basis_gates = ['u', 'cx'], optimization_level = 3) 
transp.draw(output='mpl')

In [ ]:
#-----------------------Tamanho do circuito-----------------------------

print('U:', transp.count_ops()['u']) #Portas U
print('CNOT:', transp.count_ops()['cx']) #Portas CNOT
print('Depth:', transp.depth()) #Depth
print('Portas:', transp.count_ops()['cx'] + transp.count_ops()['u']) #Total de portas
print('Q-bits:', nq + nb + 1) #Número de q-bits
print('Bits clássicos:', nb + 1) #Número de bits clássicos

In [ ]:
#-----------------------Solução do Qiskit Aqua-----------------------------

classica = np.array(linalg.solve(B, p)) #Solução clássica
aqua = HHL().solve(B, p)
aqua_vector = Statevector(aqua.state).data
nq = aqua.state.width() - nb - 1
aqua_solution = np.array(aqua_vector[2**(nq + nb): (2**(nq + nb) + 2**nb)])
aqua_solution = np.real(aqua_solution)
aqua_solution = aqua.euclidean_norm * aqua_solution / linalg.norm(aqua_solution) #Solução do Qiskit Aqua
aqua_circ = transpile(aqua.state, basis_gates = ['u', 'cx'], optimization_level = 3) #Transpilação do circuito
erro = 100 * np.mean(np.abs((aqua_solution - classica) / classica)) #Erro teórico
fidelidade = state_fidelity(aqua_solution/ linalg.norm(aqua_solution), classica / linalg.norm(classica)) #Fidelidade

#Escreve os resultados do Aqua 
print('Solução Aqua:', aqua_solution)
print('Solução Clássica:', classica) 
print('Erro teórico:', erro)
print('Fidelidade:', fidelidade)
print('U:', aqua_circ.count_ops()['u'])
print('CNOT:', aqua_circ.count_ops()['cx'])
print('Depth:', aqua_circ.depth())
print('Portas:', aqua_circ.count_ops()['cx'] + aqua_circ.count_ops()['u'])

print(aqua.state)

In [ ]:
#-----------------------Simulação ideal-----------------------------

back_sim = Aer.get_backend('qasm_simulator') #qasm_simulator bakend
job_sim = back_sim.run(transpile(transp, back_sim), shots=8192) 
result_sim = job_sim.result() 
ideal_counts = result_sim.get_counts(circuito) #Contagem
plot_histogram([ideal_counts]) #Histrograma

In [ ]:
#-----------------------Computador quântico real-----------------------------

IBMQ.save_account('API Token', overwrite=True)
provider = IBMQ.load_account()

back_ibm = provider.get_backend('ibmq_quito')
job_ibm = back_ibm.run(transpile(transp, back_ibm), shots=8192)
result_ibm = job_ibm.result()
real_counts = result_ibm.get_counts(transp) #Contagem

#Cálculo da solução real
for k, v in real_counts.items():
    if k[0] == "1" and k[1] == "1":
        menos = 2 * v / (8192 * C**2) #(teta1 - teta2)**2
    elif k[0] == "1" and k[1] == "0":                 
        mais = 2 * v / (8192 * C**2) #(teta1 + teta2)**2
        
real = np.array([(np.sqrt(menos) - np.sqrt(mais)) / 2, (np.sqrt(menos) + np.sqrt(mais)) / 2]) #Solução real

classica = np.abs(classica) #Solução clássica em módulo para calcular o erro
erro = 100 * np.mean(np.abs((real - classica) / classica)) #Erro teórico
fidelidade = state_fidelity(real/ linalg.norm(real), classica / linalg.norm(classica)) #Fidelidade

#Escreve os resultados
print('Solução clássica:', classica)
print('Solução real:', real)
print('Erro teórico:', erro, '%')
print('Fidelidade:', fidelidade)

plot_histogram([ideal_counts, real_counts], legend=['Qasm simulator', 'ibmq_quito']) #Histrograma ideal versus real